# **Exploring Analytics**
In this notebook, I want to explore the types of analytics I could calculate for Boggle performance. 

# Setup
The cells below will set up the rest of the notebook.

I'll start by configuring the kernel: 

In [1]:
# Change the working directory 
%cd ..

# Enable the autoreload extension, which will automatically load in new code as it's written
%load_ext autoreload
%autoreload 2

d:\data\programming\boggle-analytics


Now I'll import some necessary modules:

In [2]:
# General import statements
import pandas as pd
import json

# Loading and Transforming Data
Before I do any sort of analysis, I want to load in the `.xlsx` that contains my data. Once loaded, I'll add a couple of columns in that will help with the rest of the analysis.

In [3]:
# Loading in the original data
raw_boggle_data_df = pd.read_excel("data/boggle-game-records.xlsx")

# Make a copy of the original data, which we'll add new columns to
boggle_data_df = raw_boggle_data_df.copy()

# Add a "point_overlap" column, which indicates how many points Trevor and Sarah have in common
boggle_data_df["point_overlap"] = boggle_data_df.apply(
    lambda row: row.trevor_points_potential - row.trevor_points_scored, axis=1
)

# Add a "sarah_points_potential" column, which indicates how many points Sarah could have gotten
boggle_data_df["sarah_points_potential"] = boggle_data_df.apply(
    lambda row: row.sarah_points_scored + row.point_overlap, axis=1
)

# Calculating Game-Level Statistics
Now: the `boggle_data_df` are round-based statistics. 

In [4]:
# Create a new DataFrame that aggregates game-level information
game_level_stats_df = (
    boggle_data_df.groupby("game_date")
    .agg(
        trevor_total_points=("trevor_points_scored", "sum"),
        trevor_min_scoring_round=("trevor_points_scored", "min"),
        trevor_max_scoring_round=("trevor_points_scored", "max"),
        sarah_total_points=("sarah_points_scored", "sum"),
        sarah_min_scoring_round=("sarah_points_scored", "min"),
        sarah_max_scoring_round=("sarah_points_scored", "max"),
    )
    .reset_index()
)

# Add a column indicating the winner of each game
game_level_stats_df["winner"] = game_level_stats_df.apply(
    lambda row: (
        "Trevor" if row.trevor_total_points > row.sarah_total_points else "Sarah"
    ),
    axis=1,
)

# Saving Data
Now, I'm going to save the data. This will allow me to consume it within a dashboard! 

In [5]:
# Save both the round-level and game-level stats as Excel file
boggle_data_df.to_excel("data/round-level-stats.xlsx", index=False)
game_level_stats_df.to_excel("data/game-level-stats.xlsx", index=False)